# Demo: Serenity VaR Calculation API

In addition to supporting risk attribution, the latest version of Serenity also supports running VaR calculations
and backtesting those VaR models with your portfolio. This notebook will show you how to select a VaR model and
use it to compute Value at Risk (VaR) for a portfolio constructed from the Serenity security master.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2
%run var_api_helper.py
%run -i init_demo.py 

To run a VaR calculation the first thing we need is a portfolio:

In [ ]:
asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'BTC': 100,
    'ETH': 1000,
    'SOL': 10000,
    'BNB': 50000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')

The next thing we need is a model configuration. IMPORTANT: this feature is not yet supported in production, but we are including it here because this is the pattern you will need to follow going forward:

In [ ]:
from serenity_sdk.types import CalculationContext

model_short_name = 'risk.var.parametric.normal'  
model_meta = api.model().load_model_metadata(datetime.date.today())
model_config_id = model_meta.get_model_configuration_id(model_short_name)

Now that we have it we can create the `CalculationContext`, the same as with risk attribution. We will set an as-of-date and pass our parametric VaR model configuration ID. We then run VaR for a given date:

In [ ]:
from datetime import date

# construct the input parameters for VaR
ctx = CalculationContext(as_of_date=date(2020, 5, 1), model_config_id=model_config_id)

# run the VaR calc
result = api.risk().compute_var(ctx, portfolio)
result

We can also run a backtest:

In [ ]:
bt_quantiles = [1, 5, 95, 99]
result = api.risk().compute_var_backtest(ctx, portfolio, date(2020, 5, 1), date(2022, 5, 1), quantiles=bt_quantiles)

In [ ]:
# unpack into time series so that it is easier to work with
breach_count_period = 365
(baselines, 
vars_abs_by_qs, vars_rel_by_qs, 
pnls_abs, pnls_rel, 
var_breaches, rolling_breaches) = convert_to_timeseries(result, bt_quantiles, breach_count_period)

# Peeking at results

In [ ]:
# Select quantile
q = 99

run_date_col = 'run date'
base_line_col = 'baseline'
var_col = f'{q}%-VaR'
var_rel_col = f'{q}%-VaR (relative)'
pnl_col = 'P&L'
breach_col = 'Breach'
backtest_df = pd.DataFrame({
    run_date_col: pd.to_datetime(baselines.index), 
    base_line_col: baselines, 
    var_col: vars_abs_by_qs[q], 
    var_rel_col: vars_rel_by_qs[q], 
    pnl_col: pnls_abs, 
    breach_col: var_breaches[q]
}).set_index(run_date_col)


loss_fmt = lambda val: f'${val:,.2f}' if val >= 0 else f'(${abs(val):,.2f})'
pct_fmt = lambda val: f'({abs(val):,.1%})' if val < 0 else f'{abs(val):,.1%}'

red_blue_formatter = lambda val: 'color: blue' if val > 0 else 'color: red'
backtest_df.head().style.format({
  base_line_col: loss_fmt,
  var_col: loss_fmt,
  var_rel_col: pct_fmt,
  pnl_col: loss_fmt
}).applymap(red_blue_formatter, subset=[pnl_col])

# Breaches
And we can identify the breach dates:

In [ ]:
backtest_df[backtest_df[breach_col]].head().style.format({
  base_line_col: loss_fmt,
  var_col: loss_fmt,
  var_rel_col: pct_fmt,
  pnl_col: loss_fmt
}).applymap(red_blue_formatter, subset=[pnl_col])

Now let's do some charting with the backtest data; this is a sneak peek at the kinds of visualizations we will be offering in the Serenity front-end at the end of 2022:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import binom

kind = "Historical Sim"

scale_unit, scale = 'mil', 1e-6

# convert to pd.Timestamp to date time to ease plotting
dt_index = pd.to_datetime(baselines.index)

fig, axs = plt.subplots(3, 1, figsize=(14, 10), gridspec_kw={'height_ratios': [1, 2, 1]}, sharex=True)

## Portfolio Value Profile
ax0 = axs[0]
ax0.set_title(f"Portfolio Value")
ax0.set_ylabel(f"Value ($) in {scale_unit}")
ax0.plot(dt_index, scale * baselines)
ax0.xaxis.set_tick_params(labelbottom=True)


# VaR & PnL profiles
# Plot in P&L terms. So, VaR will be on the negative side. 
ax1 = axs[1]

nvars_ws = scale * vars_abs_by_qs
pnls_abs_ws = scale * pnls_abs

ax1.set_title(f"VaR Backtest ({kind})")
ax1.set_ylabel(f"PnLs ($) in {scale_unit}")

ax1.bar(dt_index, pnls_abs_ws, lw = 0.5, color="purple", label="realised PnL")
ax1.fill_between(dt_index, nvars_ws[99], nvars_ws[95], alpha=0.5, color="royalblue", label="95%-99%")
ax1.fill_between(dt_index, nvars_ws[95], nvars_ws[5], alpha=0.5, color="lightsteelblue", label="05%-95%")
ax1.fill_between(dt_index, nvars_ws[5], nvars_ws[1], alpha=0.5, color="royalblue", label="01%-05%")

for q in bt_quantiles:
    q_breaches = var_breaches[q]
    color = 'r' if q in [1,99] else 'orange'
    ax1.plot(dt_index[q_breaches], pnls_abs_ws[q_breaches], '.', color=color) #, markersize=5)

ax1.plot()
ax1.legend()
ax1.xaxis.set_tick_params(labelbottom=True)

## Now, traffic light
ax2 = axs[2]
ax2.set_title('Traffic Light Test')
ax2.set_ylabel('Rolling breach counts')

q_rolling = 99
for q in [q_rolling]: #[100-q_rolling, q_rolling]:
    line_style = '-' if q > 50 else '--'
    ax2.plot(dt_index, rolling_breaches[q], ls = line_style, color = 'purple', label=f'{q}%-VaR')
ax2.legend()

x = np.arange(0, breach_count_period)
green_amber, amber_red = [np.min(x[1 - binom.cdf(x, breach_count_period, 1.0-q_rolling/100.) < cf]) for cf in [0.05, 0.0001]]
y_max = amber_red + (amber_red - green_amber)
ax2.fill_between(dt_index, 0.0, green_amber + .4, color='g', alpha=0.1)
ax2.fill_between(dt_index, green_amber + .8, amber_red + .4, color='y', alpha=0.1)
ax2.fill_between(dt_index, amber_red + .8, y_max, color='r', alpha=0.1)
ax2.set_ylim([0.0, y_max])

plt.tight_layout()
plt.show()